### Load imports and run the no LID case:

In [9]:
# Run this block to run the SWMM model on the 6-subcatchment Wingohocking model BEFORE adding GSI
%reset
import yaml
from swmm_objects import *
from swmm_read_cso import *
from swmm_run_cso import *
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "ewri2017"
username = "arthur"
password = "jXlIkLfu5Dfr"
authenticationDatabase = "admin"
collection = 'july2008_1'
client = MongoClient(mongoServer)
swmmInpFile = "wingohocking_PWD_July.inp"  # initial SWMM .inp file
lidZeroFile = "PWD_wingo_Wakefield_list_zero.yaml"
################################################

db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database

thisRunCollection = db[collection]    # use or create the collection
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
f = open(lidZeroFile,'r')   # Read the no LID changable parameters from file
zeroLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
zeroLidDocId = runswmm(zeroLidList,swmmInpStr,thisRunCollection)  # run SWMM on the no LID model for baseline
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
zeroLidPeak = cursor['peak']
zeroLidVolume = cursor['volume']
zeroLidCSOVolume = cursor['cso_volume']
zeroLidRunoff = cursor['runoff']
print "Stored record %s for zero LID usage:  peak (CFS) = %s, volume (MGal) = %s, cso_volume = %s, runoff = %s" \
% (zeroLidDocId,zeroLidPeak,zeroLidVolume,zeroLidCSOVolume,zeroLidRunoff)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
0.0 min, 14.82 sec
volume = 183.249
Stored record 582f765bc101578498a4f9b3 for zero LID usage:  peak (CFS) = 1906.45, volume (MGal) = 183.249, cso_volume = 0.0, runoff = 565.247


### Loop ALL subcatchment LID numbers over a range:

In [2]:
# Now, apply a range of Wakefield Park rain gardens in all seven subcatchments simultaneously
from copy import deepcopy
for numberForAllLid in range(1,21):
    newRunList=deepcopy(zeroLidList)  # copy the zeroLidList into a new list
    for lid in newRunList:
        lid['Number'] = numberForAllLid  # the same number of LIDs will be in every subcatchment
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection)
    cursor = thisRunCollection.find_one({'_id':newRunDocId})
    newVolume = cursor['volume']
    volumeReduction = zeroLidVolume - newVolume
    print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"

0.0 min, 13.82 sec
volume = 183.137
Stored record 582f427ec101578498a4f99a for volume = 183.137
volumeReduction = 0.112
0.0 min, 15.30 sec
volume = 183.026
Stored record 582f428ec101578498a4f99b for volume = 183.026
volumeReduction = 0.223
0.0 min, 15.77 sec
volume = 182.915
Stored record 582f429ec101578498a4f99c for volume = 182.915
volumeReduction = 0.334
0.0 min, 19.86 sec
volume = 182.804
Stored record 582f42b2c101578498a4f99d for volume = 182.804
volumeReduction = 0.445
0.0 min, 16.50 sec
volume = 182.692
Stored record 582f42c2c101578498a4f99e for volume = 182.692
volumeReduction = 0.557
0.0 min, 16.04 sec
volume = 182.581
Stored record 582f42d2c101578498a4f99f for volume = 182.581
volumeReduction = 0.668
0.0 min, 14.62 sec
volume = 182.47
Stored record 582f42e1c101578498a4f9a0 for volume = 182.47
volumeReduction = 0.779
0.0 min, 14.56 sec
volume = 182.359
Stored record 582f42f0c101578498a4f9a1 for volume = 182.359
volumeReduction = 0.89
0.0 min, 14.36 sec
volume = 182.247
Stored 

### To display all of the runs from the database:

In [3]:
runs = db[collection]
cursor = runs.find()
for run in cursor:
    print run
    print ''

{u'lidDict': None, u'swmmStartTime': u'2016-11-18 13:02:13.807802', u'evaporation': u'62.843', u'swmmInputFileStr': u'[TITLE]\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\n[OPTIONS]\nFLOW_UNITS\tCFS \nINFILTRATION\tGREEN_AMPT \nFLOW_ROUTING\tDYNWAVE \nLINK_OFFSETS\tDEPTH \nMIN_SLOPE\t0 \nALLOW_P

### To Reconstitute the SWMM input file for any of the runs above

In [ ]:
runs = db[collection]
cursor = runs.find_one({'_id':zeroLidDocId})
#print cursor
swmmInputFileStr = cursor["swmmInputFileStr"]
f = open("SWMM_reconstituted.inp",'w')
f.write(swmmInputFileStr)  # write out the swmmInputFileStr for modified problem
f.close()